### Intel MobileODT Kaggle competition

https://www.kaggle.com/c/intel-mobileodt-cervical-cancer-screening


In [1]:
import os, sys
from __future__ import print_function, division
from importlib import reload 
import utils_p3; reload(utils_p3)
from utils_p3 import *
%matplotlib inline
from IPython.display import FileLink
import tensorflow as tf
import six
import numpy as np
import pandas as pd
import cv2
import glob
import random
from PIL import ImageFile
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
#print("TensorFlow version: %s" % tf.__version__)
print("Theano version: %s" % theano.__version__)
print("Keras version: %s" % keras.__version__)

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:01:00.0)
Using Theano backend.


Theano version: 0.9.0
Keras version: 2.0.3


### Global declarations

In [2]:
HOME_DIR = os.getcwd()
#path = "data/imgs/"
path = "data/testing/"
#path = "data/testing/sample/"
batch_size = 1 # for pre-computation of last conv layer's output and the 5 augmented copies of training (OOM issue)
#batch_size = 32 # for FCL and submission computation
ImageFile.LOAD_TRUNCATED_IMAGES = True
keras.backend.image_data_format() #verify image_data_format for theano vs TF

'channels_first'

### Setup batches

In [3]:
batches = get_batches(path+'train', batch_size=batch_size, shuffle=False)
val_batches = get_batches(path+'valid', batch_size=batch_size, shuffle=False)
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False)
steps_per_epoch = int(np.ceil(batches.samples/batch_size))
validation_steps = int(np.ceil(val_batches.samples/batch_size))

Found 5048 images belonging to 3 classes.
Found 1683 images belonging to 3 classes.
Found 4018 images belonging to 1 classes.


In [4]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 5048 images belonging to 3 classes.
Found 1683 images belonging to 3 classes.
Found 4018 images belonging to 1 classes.


## Imagenet conv features with VGG16
Based on code from Statefarm_original notebook of @Jeremy, lesson 4.

Since we have so little data, and it is similar to imagenet images (full color photos), using pre-trained VGG weights is likely to be helpful - in fact it seems likely that we won't need to fine-tune the convolutional layer weights much, if at all.

So we can pre-compute the output of the last convolutional layer, as we did in lesson 3 when we experimented with dropout. (However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.)

In [5]:
# Import our class, using VGG16 with BatchNorm
import vgg16bn_p3; reload(vgg16bn_p3) # *_p3 version code for Python 3.6 and Keras 2.0
from vgg16bn_p3 import Vgg16BN

### Step 1: pre-compute the output of the last convolutional layer

In [6]:
# Grab VGG16 and find the last convolutional layer, we try to use 512x512 size over 224x224 original VGG16.
vgg = Vgg16BN()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [7]:
# Build a new model that includes everything up to that last convolutional layer
conv_model = Sequential(conv_layers)

In [ ]:
# Predict the outputs of that model by calculating the activations of that last convolutional layer
conv_feat = conv_model.predict_generator(batches, int(np.ceil(batches.samples/batch_size)), workers=3)

In [ ]:
# As this takes time, save it to load it in the future
save_array(path+'results/bn_conv_feat.dat', conv_feat)

In [ ]:
conv_val_feat = conv_model.predict_generator(val_batches, int(np.ceil(val_batches.samples/batch_size)), workers=3)

In [ ]:
save_array(path+'results/bn_conv_val_feat.dat', conv_val_feat)

In [ ]:
conv_test_feat = conv_model.predict_generator(test_batches, int(np.ceil(test_batches.samples/batch_size)), workers=3)

In [ ]:
save_array(path+'results/bn_conv_test_feat.dat', conv_test_feat)

In [ ]:
# We can look at the original model and find the last convo layer "conv2d_13" with output shape (none, 512, 14, 14)
# and compare it with the shape of of our new model's output.
model.summary()

In [ ]:
# It's the same than "conv2d_13" !
conv_val_feat.shape

In [ ]:
# If this notebook was fully run once, we can directly reload the activations
conv_feat = load_array(path+'results/bn_conv_feat.dat')
conv_val_feat = load_array(path+'results/bn_conv_val_feat.dat')
#conv_test_feat = load_array(path+'results/bn_conv_test_feat.dat')
conv_val_feat.shape

### Step 2: Build new model on top, with dense layers
Since we've pre-computed the output of the last convolutional layer, we need to create a network that takes that as input, and predicts our 3 classes.

Let's try using a simplified version of VGG's dense layers.

In [ ]:
# we make 'p' a parameter to try different Dropout amounts
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(3, activation='softmax')
        ]

In [ ]:
p=0.5

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# quick test, verify batch_size
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
# full run on 15 epochs
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=15, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/bn_conv22.h5')

### Step 3: Pre-computed data augmentation by making 5 augmented copies of training set

We'll use our usual data augmentation parameters:

In [10]:
# Warning: uses HUGE amount of RAM (up to 60gb) and takes 90 mins for a simple 'batches.samples*2'
# with batch_size=64. Check forum at
# http://forums.fast.ai/t/state-farm-full-how-not-to-run-out-of-memory-with-vgg-da-batches-samples-5/3469/2
# Maybe reduce batch_size and workers to 1 ? YES !!!
# Also kernel needs a 'Restart and clear output' to clear RAM (now 15gb RAM + 12gb SWAP used),
# so run Step 1 + 2 first, then reset and run Step 3.
gen_t = image.ImageDataGenerator(rotation_range=8, height_shift_range=0.025,
                                shear_range=0.05, channel_shift_range=10, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=1, shuffle=False)

Found 5048 images belonging to 3 classes.


In [11]:
da_conv_feat = conv_model.predict_generator(da_batches, int(np.ceil(da_batches.samples*5)), workers=3)

In [12]:
save_array(path+'results/bn_da5_conv_feat.dat', da_conv_feat)

Let's include the real training data as well in its non-augmented form.

In [13]:
conv_feat = load_array(path+'results/bn_conv_feat.dat')

In [ ]:
da_conv_feat = load_array(path+'results/bn_da5_conv_feat.dat')

In [14]:
da_conv_feat_update = np.concatenate([da_conv_feat, conv_feat])

In [15]:
save_array(path+'results/bn_da5_conv_feat_update.dat', da_conv_feat_update)

In [ ]:
da_conv_feat_update = load_array(path+'results/bn_da5_conv_feat_update.dat')

Since we've now got a dataset 6x bigger than before, we'll need to copy our labels 6 times too.

In [16]:
da_trn_labels = np.concatenate([trn_labels]*6)

In [17]:
save_array(path+'results/bn_da5_trn_labels.dat', da_trn_labels)

In [ ]:
da_trn_labels = load_array(path+'results/bn_da5_trn_labels.dat')

Based on some experiments the previous model works well, maybe with bigger dense layers like 512 later ?

In [18]:
# we make 'p' a parameter to try different Dropout amounts
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(3, activation='softmax')
        ]

In [42]:
p=0.6

In [43]:
bn_da_model = Sequential(get_bn_da_layers(p))
bn_da_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
conv_val_feat = load_array(path+'results/bn_conv_val_feat.dat')

In [44]:
# quick test, verify batch_size
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=32, epochs=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 30288 samples, validate on 1683 samples
Epoch 1/1
30288/30288 [==============================] - 58s - loss: 1.4089 - acc: 0.5964 - val_loss: 1.3028 - val_acc: 0.5573


In [45]:
# full run on 15 epochs
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=32, epochs=50, 
             validation_data=(conv_val_feat, val_labels))

Train on 30288 samples, validate on 1683 samples
Epoch 1/50
30288/30288 [==============================] - 27s - loss: 0.8133 - acc: 0.7295 - val_loss: 1.1924 - val_acc: 0.6554
Epoch 2/50
30288/30288 [==============================] - 28s - loss: 0.6449 - acc: 0.7860 - val_loss: 1.2107 - val_acc: 0.6649
Epoch 3/50
30288/30288 [==============================] - 27s - loss: 0.5768 - acc: 0.8185 - val_loss: 1.1218 - val_acc: 0.6863
Epoch 4/50
30288/30288 [==============================] - 27s - loss: 0.4990 - acc: 0.8430 - val_loss: 1.3277 - val_acc: 0.6821
Epoch 5/50
30288/30288 [==============================] - 27s - loss: 0.4303 - acc: 0.8627 - val_loss: 1.3184 - val_acc: 0.6833
Epoch 6/50
30288/30288 [==============================] - 25s - loss: 0.3896 - acc: 0.8765 - val_loss: 1.2210 - val_acc: 0.7106
Epoch 7/50
30288/30288 [==============================] - 25s - loss: 0.3702 - acc: 0.8862 - val_loss: 1.2278 - val_acc: 0.7011
Epoch 8/50
30288/30288 [==============================]

In [ ]:
# Let's save those weights.
bn_da_model.save_weights(path+'models/bn_da5_conv22_50e.h5')

In [ ]:
# Let's load those weights.
bn_da_model.load_weights(path+'models/bn_da5_conv22_50e.h5')

#### some test not saved with different learning rates or optimizer

In [ ]:
bn_da_model.compile(RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
from keras.optimizers import *
bn_da_model.compile(Adagrad(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_da_model.compile(Adadelta(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_da_model.compile(Adamax(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_da_model.compile(Nadam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

### Submissions

Don't forget to add clipping for Kaggle submissions as it's very important to get the best cross_entropy loss function.

In [46]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [47]:
val_preds = bn_da_model.predict(conv_val_feat, batch_size=batch_size)

In [48]:
np.mean(keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval())

0.88522841921048512

In [49]:
conv_test_feat = load_array(path+'results/bn_conv_test_feat.dat')

In [50]:
preds = bn_da_model.predict(conv_test_feat, batch_size=batch_size)

In [51]:
subm = do_clip(preds,0.93)

In [55]:
subm_name = path+'results/subm_22_bn_da_vgg_50e.csv'

In [56]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [57]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image_name', [a[8:] for a in test_filenames])
submission.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.007778,0.1044,0.893303
1,1.jpg,0.007778,0.9300,0.007778
2,10.jpg,0.007778,0.9300,0.007778
3,100.jpg,0.007778,0.9300,0.007778
4,10000.jpg,0.007778,0.9300,0.007778


In [58]:
submission.to_csv(subm_name, index=False)

In [59]:
FileLink(subm_name)

/media/eric/SSD500/fastai/deeplearning1/nbs/MobileODT/data/testing/results/subm_22_bn_da_vgg_50e.csv